In [60]:
suppressMessages(library(rwwa))

mdl_args <- list("hdwi_chile2026" = list(dist = "gev", type = "shiftscale", covnm = "gmst", lower = F),
                 "hdwi_patagonia" = list(dist = "gev", type = "shift", covnm = "gmst", lower = F),
                 "pr-ndj_chile2026" = list(dist = "norm_logt", type = "shift", covnm = "gmst", lower = T),
                 "pr-ndj_patagonia" = list(dist = "norm_logt", type = "shift", covnm = "gmst", lower = T))

rp <- list("hdwi" = 5, "pr-ndj" = 5)

nsamp <- 1000

png_res <- 240

# CORDEX

## GMST only

In [61]:
varnm <- "pr-ndj"
for (rnm in c("chile2026", "patagonia")) {
    
    fl <- list.files("ts-cordex", pattern = paste0(varnm, "_",rnm), full.names = T)
    
    for (fnm in fl) {
        print(fnm)
    
        # if (fnm %in% c("ts-cordex/mexico-floods_rx3day_CAM-44_CNRM-CM5_r1i1p1_CRCM5.dat")) next

        res_fnm <- gsub("sam-fires_","",gsub(varnm, paste0(varnm,"_gmst-only"),gsub(".dat", ".csv", gsub("ts-cordex/", "res-cordex/res-cordex_", fnm))))
        if (exists(res_fnm)) next
    
        gcm <- gsub("_", "_rcp85_", gsub("i1p1","",paste0(strsplit(fnm, "_")[[1]][5:6], collapse = "_")))
    
        gmst_fnm <- list.files("~/00_WWA_project_folder/live/data/cmip5_gmsts", paste0("gsat_",gcm, ".dat"), full.names = T)
        if (length(gmst_fnm) == 0) next

        new_gmst_fnm <- paste0("ts-cordex/",basename(gmst_fnm))

        if (!file.exists(new_gmst_fnm)) file.copy(gmst_fnm, new_gmst_fnm)
    
        # nino <- load_ts(nino_fnm, col.names = c("year", "nino34"))
        gmst <- load_ts(new_gmst_fnm, col.names = c("year", "gmst"))
        ts <- load_ts(fnm, col.names = c("year", "x"))
        df <- merge(gmst, ts)

        # trend fitting
        mdl <- do.call(fit_ns, append(mdl_args[[paste0(varnm,"_",rnm)]], list("data" = df, "varnm" = "x")))
    
        # use model's 2023 GMST & observed Nino to define factual climate
        gmst_2025 <- df$gmst[df$year == 2026]
        
        cov_2025 <- data.frame("gmst" = gmst_2025)
        cov_cf <- data.frame("gmst" = gmst_2025 - 1.3)
        cov_fut <- data.frame("gmst" = gmst_2025 + 1.3)
    
        # bootstrap results
        res <- cmodel_results(mdl, rp = rp[[varnm]], cov_f = cov_2025, cov_hist = cov_cf, cov_fut = cov_fut,
                              y_now = 2026, y_start = 1979, y_fut = 2070, nsamp = nsamp)
    
        # # bootstrap Nino amplitude & correlation between Nino & precip in this region
        # set.seed(42)
        # nino_boot <- sapply(1:nsamp, function(i) {
        #     boot_df <- nino_clim[sample(1:nrow(nino_clim), replace = T), ]
        #     c("nino_cor" = cor(boot_df$nino34, boot_df$pr), "nino_sd" = sd(boot_df$nino34))
        # })
        # nino_qq <- rbind("est" = c(cor(df$nino34, df$pr), sd(df$nino34)), apply(nino_boot,1,quantile,c(0.025, 0.975)))
        # nino_qq <- t(data.frame(unlist(sapply(colnames(nino_qq), function(cnm) {
        #     setNames(nino_qq[,cnm], paste0(cnm,"_", c("est", "lower", "upper")))
        # }, simplify = F, USE.NAMES = F))))
                                           
        # res <- cbind(res, nino_qq)
        write.csv(res, res_fnm)
    
        # quick return level plots to check fit
        rlplot_fnm <- gsub("csv", "png", gsub("res-cordex/res-cordex", "mdl-eval/rlplot_", res_fnm))
        if(!file.exists(rlplot_fnm)) {
            png(rlplot_fnm, h = 480, w = 480 * 2); par(cex.main = 1.4, mfrow = c(1,2), oma = c(0,0,2,0)); {
                plot_returnlevels(mdl, cov_f = cov_2025, cov_cf = cov_cf, nsamp = 100, main = "Present vs PI",legend_pos = "bottomright")
                mtext(outer = T, gsub(".dat","",paste0(strsplit(fnm, "_")[[1]][3:5], collapse = "_")), font = 2, cex = 1.5)
            }; dev.off()
        }
    }
}

[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-22_HadGEM2-ES_r1i1p1_RegCM4-7.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-22_HadGEM2-ES_r1i1p1_REMO2015.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-22_MPI-ESM-LR_r1i1p1_REMO2015.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-22_MPI-ESM-MR_r1i1p1_RegCM4-7.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-22_NorESM1-M_r1i1p1_RegCM4-7.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-22_NorESM1-M_r1i1p1_REMO2015.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-44_CanESM2_r1i1p1_RCA4.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-44_CNRM-CM5_r1i1p1_RCA4.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-44_CSIRO-Mk3-6-0_r1i1p1_RCA4.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-44_EC-EARTH_r12i1p1_RCA4.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-44_GFDL-ESM2M_r1i1p1_RCA4.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-44_HadGEM2-ES_r1i1p1_RCA4.dat"
[1] "ts-cordex/sam-fires_pr-ndj_chile2026_SAM-44_HadGEM2-

In [62]:
mtype <- "gmst-only"
varnm <- "pr-ndj"

for (rnm in c("chile2026", "patagonia")) {
    print(rnm)
    mdl_res <- t(sapply(list.files("res-cordex", pattern = paste0(c(varnm, mtype, rnm), collapse = "_"), full.names = T), read.csv))
    rownames(mdl_res) <- apply(sapply(strsplit(gsub(".csv","",rownames(mdl_res)), "_"), "[", 5:8), 2, paste0, collapse = "_")
    write.csv(mdl_res, paste0("res-cordex_",varnm,"_",mtype,"_",rnm,".csv"))
    }

[1] "chile2026"
[1] "patagonia"
